# Cobrapy
Check whether the model (.xml) is validated for applying to Cobrapy (https://github.com/opencobra/cobrapy)

In [1]:
import numpy as np
import cobra
import os
from os.path import join

from pprint import pprint
from cobra.io import validate_sbml_model
report = validate_sbml_model(filename='iMM904.xml') # (2009) https://pubmed.ncbi.nlm.nih.gov/19321003/
pprint(report)

(<Model iMM904 at 0x1438e1b90>,
 {'COBRA_CHECK': [],
  'COBRA_ERROR': [],
  'COBRA_FATAL': [],
  'COBRA_WARNING': [],
  'SBML_ERROR': [],
  'SBML_FATAL': [],
  'SBML_SCHEMA_ERROR': [],
  'SBML_WARNING': []})


In [2]:
os.getcwd()

'/Users/sungminhwang'

# Load a model (.xml file)

In [3]:
from cobra.io import read_sbml_model
model= read_sbml_model('iMM904.xml')
model

# These reations are not found from Bigg: 
# ALCD2x_copy1, ALCD2x_copy2 ,CERH124_copy1, CERH126_copy1, CERH124_copy2, CERH126_copy2, GCC2cm_copy1, GCC2cm_copy2

Name,iMM904
Memory address,10a88da50
Number of metabolites,1226
Number of reactions,1577
Number of genes,905
Number of groups,0
Objective expression,1.0*BIOMASS_SC5_notrace - 1.0*BIOMASS_SC5_notrace_reverse_93090
Compartments,"cytosol, extracellular space, mitochondria, peroxisome/glyoxysome, endoplasmic reticulum, vacuole, golgi apparatus, nucleus"


In [4]:
######################## Delete !!


from cobra.io import read_sbml_model
from cobra.flux_analysis import single_reaction_deletion
import pandas as pd


#model.genes.get_by_id('YHR104W')
model.reactions.get_by_id('13GS')
#model.reactions


df = model.reactions.get_by_id('13GS')
df_11 = pd.DataFrame(df, columns= ['Reactions'])
#df_11.to_csv('test.csv', index = False)

ValueError: DataFrame constructor not properly called!

# Check what reactions/metabolites/genes are present in the model

In [5]:
from cameo import fba
import pandas as pd

# reactions from iMM904
model_rxn = model.reactions
model_rxn_df = pd.DataFrame(model_rxn, columns= ['Reactions'])
#model_rxn_df.to_csv('iMM904_rxn.csv', index = False)

In [6]:
# metabolites from iMM904
model_met = model.metabolites
model_met_df = pd.DataFrame(model_met, columns= ['Metabolites'])
#model_met_df.to_csv('iMM904_met.csv', index = False)

In [7]:
# genes from iMM904
model_gene = model.genes
model_gene_df = pd.DataFrame(model_gene, columns=['Genes'])
#model_gene_df.to_csv('iMM904_genes.csv', index = False)                                                

In [8]:
model.medium

{'EX_fe2_e': 999999.0,
 'EX_glc__D_e': 10.0,
 'EX_h2o_e': 999999.0,
 'EX_h_e': 999999.0,
 'EX_k_e': 999999.0,
 'EX_na1_e': 999999.0,
 'EX_so4_e': 999999.0,
 'EX_nh4_e': 999999.0,
 'EX_o2_e': 2.0,
 'EX_pi_e': 999999.0}

# BY4741: MATa his3Δ1 leu2Δ0 met15Δ0 ura3Δ0 <br>

BY4741: MATa his3Δ1 leu2Δ0 met15Δ0 ura3Δ0 <br>

Modify medium for the simulation based on Table A1. https://doi.org/10.1371/journal.pcbi.1005728
1) Glucose only <br>
2) Xylose only <br>
3) Acetate only <br>
4) Glu + Acet <br>
5) Xyl + Acet <br>

# Construction of BY4741 from iMM904

In [1]:
from cobra.io import read_sbml_model
model_iMM904_BY4741 = read_sbml_model('iMM904.xml')
model_iMM904_BY4741.solver

model_iMM904_BY4741.genes.YCL018W.knock_out() #LEU2, BY4741
model_iMM904_BY4741.genes.YEL021W.knock_out() #URA3, BY4741
model_iMM904_BY4741.genes.YLR303W.knock_out() #MET17, BY4741
#model_iMM904_BY4741.genes.YOR202W.knock_out() #HIS3: if the gene is deleted, cells cannot grow.
model_iMM904_BY4741.reactions.XYLR.knock_out() # xylose reductase (YHR104W) pathway was deleted!! h_c + nadph_c + xyl__D_c --> nadp_c + xylt_c

# test below
#model_iMM904_BY4741.reactions.ALCD2x_copy1.knock_out()  # etoh_c + nad_c <=> acald_c + h_c + nadh_c
#model_iMM904_BY4741.reactions.ALCD2x_copy2.knock_out()

Scaling...
 A: min|aij| =  1.000e+00  max|aij| =  1.000e+00  ratio =  1.000e+00
Problem data seem to be well scaled


In [2]:
# modify medium for the simulation based on Table A1. https://doi.org/10.1371/journal.pcbi.1005728
# unit (mmol/gDW/h): 1 mol of glucose every 24h to 1 gram of bacteria you might set the upper exchange flux for glucose to "1 mol / [1 gDW * 24 h]" (https://cobrapy.readthedocs.io/en/latest/media.html)

medium_iMM904_BY4741 = model_iMM904_BY4741.medium

medium_iMM904_BY4741["EX_ura_e"]= 1000    ### uracil[e] =>
medium_iMM904_BY4741["EX_ade_e"]= 0   # adenine[e] => 
medium_iMM904_BY4741["EX_cit_e"]= 0   # citrate[e] => 
medium_iMM904_BY4741["EX_gly_e"]= 0   # L-glycine[e] => 
medium_iMM904_BY4741["EX_ala__L_e"]= 0   # L-alanine[e] => 
medium_iMM904_BY4741["EX_arg__L_e"]= 0   # L-arginine[e] => 
medium_iMM904_BY4741["EX_asn__L_e"]= 0   # L-asparagine[e] => 
medium_iMM904_BY4741["EX_asp__L_e"]= 0   # L-aspartate[e] => 
medium_iMM904_BY4741["EX_cys__L_e"]= 0   # L-cysteine[e] => 
medium_iMM904_BY4741["EX_glu__L_e"]= 0   # L-glutamate[e] => 
medium_iMM904_BY4741["EX_gln__L_e"]= 0   # L-glutamine[e] => 
medium_iMM904_BY4741["EX_ile__L_e"]= 0   # L-isoleucine[e] => 
medium_iMM904_BY4741["EX_leu__L_e"]= 1000   ### L-leucine[e] => 
medium_iMM904_BY4741["EX_lys__L_e"]= 0   # L-lysine[e] => 
medium_iMM904_BY4741["EX_met__L_e"]= 0   # L-methionine[e] => 
medium_iMM904_BY4741["EX_phe__L_e"]= 0   # L-phenylalanine[e] => 
medium_iMM904_BY4741["EX_pro__L_e"]= 0   # L-proline[e] => 
medium_iMM904_BY4741["EX_ser__L_e"]= 0   # L-serine[e] => 
medium_iMM904_BY4741["EX_thr__L_e"]= 0   # L-threonine[e] => 
medium_iMM904_BY4741["EX_trp__L_e"]= 0   # L-tryptophan[e] => 
medium_iMM904_BY4741["EX_tyr__L_e"]= 0   # L-tyrosine[e] => 
medium_iMM904_BY4741["EX_val__L_e"]= 0   # L-valine[e] => 
medium_iMM904_BY4741["EX_ribflv_e"]= 0   # riboflavin[e] => 
medium_iMM904_BY4741["EX_pnto__R_e"]= 0.78   # (R)-pantothenate[e] => 
medium_iMM904_BY4741["EX_4abz_e"]= 0.78   # 4-aminobenzoate[e] => 
medium_iMM904_BY4741["EX_btn_e"]= 0.78   # biotin[e] => 
medium_iMM904_BY4741["EX_inost_e"]= 0.78   # myo-inositol[e] => 
medium_iMM904_BY4741["EX_nac_e"]= 0.78   # nicotinate[e] => 
medium_iMM904_BY4741["EX_thmpp_e"]= 0.78   # thiamine[e] => 
medium_iMM904_BY4741["EX_nh4_e"]= 1000   # ammonium[e] <=> 
medium_iMM904_BY4741["EX_fe2_e"]= 1000   # iron(2+)[e] <=> 
medium_iMM904_BY4741["EX_pi_e"]= 1000   # phosphate[e] <=> 
medium_iMM904_BY4741["EX_k_e"]= 1000   # potassium[e] <=> 
medium_iMM904_BY4741["EX_na1_e"]= 1000   # sodium[e] <=> 
medium_iMM904_BY4741["EX_so4_e"]= 1000   # sulphate[e] <=> 
medium_iMM904_BY4741["EX_h2o_e"]= 1000  # H2O
medium_iMM904_BY4741["EX_h_e"]= 1000  #H+
#not present: medium_iMM904_BY4741["r_2028"]= 0.78   # pyridoxine[e] => 
#not present: medium_iMM904_BY4741["r_1792"]= 0   # folate[e] => 
#not present: medium_iMM904_BY4741["r_4593"]= 1000   # chloride[e] <=> 
#not present: medium_iMM904_BY4741["r_4597"]= 1000   # Mg(2+)[e] <=> 
#not present: medium_iMM904_BY4741["r_4600"]= 1000   # Ca(2+)[e] <=> 
medium_iMM904_BY4741["EX_o2_e"]= 1000   # oxygen[e] <=>      # if anaerobic, set 0

#1) Glucose only <br>
#2) Xylose only <br>
#3) Acetate only <br>
#4) Glu + Acet <br>
#5) Xyl + Acet <br>
medium_iMM904_BY4741["EX_glc__D_e"]= 0   # D-glucose[e] <=> 
medium_iMM904_BY4741["EX_xyl__D_e"]= 20    # D-xylose[e] => 
medium_iMM904_BY4741["EX_ac_e"]=     0    # acetate[e] => 

model_iMM904_BY4741.medium = medium_iMM904_BY4741
model_iMM904_BY4741.medium


from cameo import fba
import pandas as pd

fba_result_iMM904_BY4741 = fba(model_iMM904_BY4741)
fba_result_iMM904_BY4741
#fba_result_iMM904_BY4741.data_frame
#fba_result_iMM904_BY4741.data_frame.to_csv('1_BY4741_5_xyl_acet.csv')

In [11]:
# change objective

#fba_result_iMM904_BY4741
model_iMM904_BY4741.objective = "ALCD2ir" # acald_c + h_c + nadh_c --> etoh_c + nad_c
model_iMM904_BY4741.optimize().objective_value

999999.0

# Visualization

In [12]:
import ipywidgets as widgets
from IPython.display import display
import cobra
import escher

solution = model_iMM904_BY4741.optimize()
escher.Builder('iMM904.Central carbon metabolism', reaction_data = solution.fluxes)

Builder(reaction_data={'CITtcp': 0.0, '13BGH': 0.0, '13BGHe': 0.0, '13GS': 4.267626304010187, '16GS': 0.0, '23…

In [13]:
#model_iMM904_BY4741.reactions.get_by_id('ALCD2ir').flux
#model_iMM904_BY4741.reactions.get_by_id('PPA').flux
model_iMM904_BY4741.reactions.get_by_id('G6PDH2r')

#model_iMM904_BY4741.genes.get_by_id('YNL241C') # G6PDH2r

Reaction identifier,G6PDH2r
Name,Glucose 6-phosphate dehydrogenase
Memory address,0x14ada0c10
Stoichiometry,"g6p_c + nadp_c --> 6pgl_c + h_c + nadph_c D-Glucose 6-phosphate + Nicotinamide adenine dinucleotide phosphate --> 6-phospho-D-glucono-1,5-lactone + H+ + Nicotinamide adenine dinucleotide phosphate - reduced"
GPR,YNL241C
Lower bound,0.0
Upper bound,999999.0


In [14]:
import pandas as pd
from cameo import fba
from cobra.flux_analysis import flux_variability_analysis

flux_variability_analysis(model_iMM904_BY4741)
table_FVA_flux_BY4741 = flux_variability_analysis(model_iMM904_BY4741)
#table_FVA_flux_BY4741.to_csv('1_FVA_5_xyl_acet.csv')

# XAE: BY4741 ∆gre3::xylA*3-XKS1 ∆pho13::xylA*3-Tal1 ∆PMR1(G681A) ∆ASC1(Q237stop) [ALE on xylose and acetate] <br>

BY4741: MATa his3Δ1 leu2Δ0 met15Δ0 ura3Δ0 <br>
XAE: BY4741 ∆gre3::xylA*3-XKS1 ∆pho13::xylA*3-Tal1 ∆PMR1(G681A) ∆ASC1(Q237stop)<br>
1) Glucose only <br>
2) Xylose only <br>
3) Acetate only <br>
4) Glu + Acet <br>
5) Xyl + Acet <br>

In [47]:
from cobra.io import read_sbml_model
model_iMM904_XAE = read_sbml_model('iMM904.xml')
model_iMM904_XAE.solver

model_iMM904_XAE.genes.YCL018W.knock_out() #LEU2, BY4741
model_iMM904_XAE.genes.YEL021W.knock_out() #URA3, BY4741
model_iMM904_XAE.genes.YLR303W.knock_out() #MET17, BY4741
model_iMM904_XAE.reactions.XYLR.knock_out() # xylose reductase (YHR104W) pathway was deleted!! h_c + nadph_c + xyl__D_c --> nadp_c + xylt_c

model_iMM904_XAE.genes.YHR104W.knock_out() #GRE3, XAE
# PHO13 (YDL236W) is not present in iMM904 model. H2O [cytoplasm] + 4-nitrophenyl phosphate [cytoplasm] --> H+ [cytoplasm] + phosphate [cytoplasm] + 4-nitrophenol [cytoplasm]
# PMR1 (YGL167C) is not present in iMM904 model. H2O [Golgi] + Ca(2+) [cytoplasm] + ATP [Golgi] --> H+ [Golgi] + phosphate [Golgi] + Ca(2+) [Golgi] + ADP [Golgi]
# ASC1 (YMR116C) is not present in iMM904 model. required to prevent frameshifting at ribosomes stalled at repeated CGA codons (https://www.yeastgenome.org/locus/S000004722)

# XylA (r_8001) : xyl__D_c ⇌ xylu__D_c
from cobra import Reaction
reaction1 = Reaction('r_8001')
reaction1.name = 'Xylose isomerase'
reaction1.lower_bound = - 1000
reaction1.upper_bound = 1000
reaction1.add_metabolites({model_iMM904_XAE.metabolites.get_by_id("xyl__D_c"): - 1.0, 
                           model_iMM904_XAE.metabolites.get_by_id("xylu__D_c"): 1.0})
reaction1.gene_reaction_rule = 'XylA'
model_iMM904_XAE.add_reaction(reaction1)

# Overexpression target of TAL1 (YGR043C and YLR354C) is present in iMM904 model.  
# Overexpression target of XKS1 (YGR194C) is present in iMM904 model.  

In [75]:
# modify medium for the simulation based on Table A1. https://doi.org/10.1371/journal.pcbi.1005728
# unit (mmol/gDW/h): 1 mol of glucose every 24h to 1 gram of bacteria you might set the upper exchange flux for glucose to "1 mol / [1 gDW * 24 h]" (https://cobrapy.readthedocs.io/en/latest/media.html)

medium_iMM904_XAE = model_iMM904_XAE.medium

medium_iMM904_XAE["EX_ura_e"]= 1000    ### uracil[e] =>
medium_iMM904_XAE["EX_ade_e"]= 0   # adenine[e] => 
medium_iMM904_XAE["EX_cit_e"]= 0   # citrate[e] => 
medium_iMM904_XAE["EX_gly_e"]= 0   # L-glycine[e] => 
medium_iMM904_XAE["EX_ala__L_e"]= 0   # L-alanine[e] => 
medium_iMM904_XAE["EX_arg__L_e"]= 0   # L-arginine[e] => 
medium_iMM904_XAE["EX_asn__L_e"]= 0   # L-asparagine[e] => 
medium_iMM904_XAE["EX_asp__L_e"]= 0   # L-aspartate[e] => 
medium_iMM904_XAE["EX_cys__L_e"]= 0   # L-cysteine[e] => 
medium_iMM904_XAE["EX_glu__L_e"]= 0   # L-glutamate[e] => 
medium_iMM904_XAE["EX_gln__L_e"]= 0   # L-glutamine[e] => 
medium_iMM904_XAE["EX_ile__L_e"]= 0   # L-isoleucine[e] => 
medium_iMM904_XAE["EX_leu__L_e"]= 1000   ### L-leucine[e] => 
medium_iMM904_XAE["EX_lys__L_e"]= 0   # L-lysine[e] => 
medium_iMM904_XAE["EX_met__L_e"]= 0   # L-methionine[e] => 
medium_iMM904_XAE["EX_phe__L_e"]= 0   # L-phenylalanine[e] => 
medium_iMM904_XAE["EX_pro__L_e"]= 0   # L-proline[e] => 
medium_iMM904_XAE["EX_ser__L_e"]= 0   # L-serine[e] => 
medium_iMM904_XAE["EX_thr__L_e"]= 0   # L-threonine[e] => 
medium_iMM904_XAE["EX_trp__L_e"]= 0   # L-tryptophan[e] => 
medium_iMM904_XAE["EX_tyr__L_e"]= 0   # L-tyrosine[e] => 
medium_iMM904_XAE["EX_val__L_e"]= 0   # L-valine[e] => 
medium_iMM904_XAE["EX_ribflv_e"]= 0   # riboflavin[e] => 
medium_iMM904_XAE["EX_pnto__R_e"]= 0.78   # (R)-pantothenate[e] => 
medium_iMM904_XAE["EX_4abz_e"]= 0.78   # 4-aminobenzoate[e] => 
medium_iMM904_XAE["EX_btn_e"]= 0.78   # biotin[e] => 
medium_iMM904_XAE["EX_inost_e"]= 0.78   # myo-inositol[e] => 
medium_iMM904_XAE["EX_nac_e"]= 0.78   # nicotinate[e] => 
medium_iMM904_XAE["EX_thmpp_e"]= 0.78   # thiamine[e] => 
medium_iMM904_XAE["EX_nh4_e"]= 1000   # ammonium[e] <=> 
medium_iMM904_XAE["EX_fe2_e"]= 1000   # iron(2+)[e] <=> 
medium_iMM904_XAE["EX_pi_e"]= 1000   # phosphate[e] <=> 
medium_iMM904_XAE["EX_k_e"]= 1000   # potassium[e] <=> 
medium_iMM904_XAE["EX_na1_e"]= 1000   # sodium[e] <=> 
medium_iMM904_XAE["EX_so4_e"]= 1000   # sulphate[e] <=> 
medium_iMM904_XAE["EX_h2o_e"]= 1000  # H2O
medium_iMM904_XAE["EX_h_e"]= 1000  #H+
#not present: medium_iMM904_XAE["r_2028"]= 0.78   # pyridoxine[e] => 
#not present: medium_iMM904_XAE["r_1792"]= 0   # folate[e] => 
#not present: medium_iMM904_XAE["r_4593"]= 1000   # chloride[e] <=> 
#not present: medium_iMM904_XAE["r_4597"]= 1000   # Mg(2+)[e] <=> 
#not present: medium_iMM904_XAE["r_4600"]= 1000   # Ca(2+)[e] <=> 
medium_iMM904_XAE["EX_o2_e"]= 1000   # oxygen[e] <=>      # if anaerobic, set 0

#1) Glucose only <br>
#2) Xylose only <br>
#3) Acetate only <br>
#4) Glu + Acet <br>
#5) Xyl + Acet <br>
medium_iMM904_XAE["EX_glc__D_e"]= 0   # D-glucose[e] <=> 
medium_iMM904_XAE["EX_xyl__D_e"]= 0    # D-xylose[e] => 
medium_iMM904_XAE["EX_ac_e"]=     2    # acetate[e] => 

model_iMM904_XAE.medium = medium_iMM904_XAE
model_iMM904_XAE.medium

from cameo import fba
import pandas as pd

fba_result_iMM904_XAE = fba(model_iMM904_XAE)
fba_result_iMM904_XAE
#fba_result_iMM904_XAE.data_frame
#fba_result_iMM904_XAE.data_frame.to_csv('2_XAE_5_xyl_acet.csv')


In [82]:
model_iMM904_XAE.reactions.get_by_id('ACtr').flux

model_iMM904_XAE.genes.get_by_id('YCR010C') #ADY2, Rxn: ACtr
#model_iMM904_XAE.genes.get_by_id('YNR002C') #ATO2, not present in the model
model_iMM904_XAE.genes.get_by_id('YKL217W') #JEN1, Rxn: L_LACt2r, PYRt2, D_LACt2
model_iMM904_XAE.genes.get_by_id('YAR015W') #ADE1, Rxn: PRASCSi
model_iMM904_XAE.genes.get_by_id('YLR359W') #ADE13, Rxn: ADSL2r, ADSL1r
model_iMM904_XAE.genes.get_by_id('YMR120C') #ADE17, Rxn: AICART, IMPC

Gene identifier,YMR120C
Name,ADE17
Memory address,0x0178031290
Functional,True
In 2 reaction(s),"AICART, IMPC"


In [68]:
# change objective

#fba_result_iMM904_XAE
model_iMM904_XAE.objective = "ALCD2ir" # acald_c + h_c + nadh_c --> etoh_c + nad_c
model_iMM904_XAE.optimize().objective_value

999999.0

In [53]:
import ipywidgets as widgets
from IPython.display import display
import cobra
import escher

solution = model_iMM904_XAE.optimize()
escher.Builder('iMM904.Central carbon metabolism', reaction_data = solution.fluxes)


Builder(reaction_data={'CITtcp': 0.0, '13BGH': 0.0, '13BGHe': 0.0, '13GS': 3.5737033277483627, '16GS': 0.0, '2…

In [169]:
import pandas as pd
from cameo import fba
from cobra.flux_analysis import flux_variability_analysis

flux_variability_analysis(model_iMM904_XAE)
table_FVA_flux_XAE = flux_variability_analysis(model_iMM904_XAE)
#table_FVA_flux_XAE.to_csv('2_XAE_FVA_5_xyl_acet.csv')

In [145]:
# change objective

#model_iMM904_XAE.objective = "ALCD2ir"
#model_iMM904_XAE.optimize().objective_value

# XAE-A1: XAE ∆gpd2::CoAdhE/SeACS ∆ald6 
BY4741: MATa his3Δ1 leu2Δ0 met15Δ0 ura3Δ0 <br/>
XAE: BY4741 ∆gre3::xylA*3-XKS1 ∆pho13::xylA*3-Tal1 ∆PMR1(G681A) ∆ASC1(Q237stop)<br>
XAE-A1: XAE ∆gpd2::CoAdhE/SeACS ∆ald6

1) Glucose only <br>
2) Xylose only <br>
3) Acetate only <br>
4) Glu + Acet <br>
5) Xyl + Acet <br>

In [54]:
from cobra.io import read_sbml_model
model_iMM904_XAE_A1 = read_sbml_model('iMM904.xml')
model_iMM904_XAE_A1.solver

model_iMM904_XAE_A1.genes.YCL018W.knock_out() #LEU2, BY4741
model_iMM904_XAE_A1.genes.YEL021W.knock_out() #URA3, BY4741
model_iMM904_XAE_A1.genes.YLR303W.knock_out() #MET17, BY4741
model_iMM904_XAE.reactions.XYLR.knock_out() # xylose reductase (YHR104W) pathway was deleted!! h_c + nadph_c + xyl__D_c --> nadp_c + xylt_c

model_iMM904_XAE_A1.genes.YHR104W.knock_out() #GRE3, XAE
# PHO13 (YDL236W) is not present in iMM904 model. H2O [cytoplasm] + 4-nitrophenyl phosphate [cytoplasm] --> H+ [cytoplasm] + phosphate [cytoplasm] + 4-nitrophenol [cytoplasm]
# PMR1 (YGL167C) is not present in iMM904 model. H2O [Golgi] + Ca(2+) [cytoplasm] + ATP [Golgi] --> H+ [Golgi] + phosphate [Golgi] + Ca(2+) [Golgi] + ADP [Golgi]
# ASC1 (YMR116C) is not present in iMM904 model. required to prevent frameshifting at ribosomes stalled at repeated CGA codons (https://www.yeastgenome.org/locus/S000004722)

model_iMM904_XAE_A1.genes.YOL059W.knock_out() #GPD2, XAE-A1
model_iMM904_XAE_A1.genes.YPL061W.knock_out() #ALD6, XAE-A1



# XylA (r_8001) : xyl__D_c ⇌ xylu__D_c
from cobra import Reaction
reaction1 = Reaction('r_8001')
reaction1.name = 'Xylose isomerase'
reaction1.lower_bound = - 1000
reaction1.upper_bound = 1000
reaction1.add_metabolites({model_iMM904_XAE_A1.metabolites.get_by_id("xyl__D_c"): - 1.0, 
                           model_iMM904_XAE_A1.metabolites.get_by_id("xylu__D_c"): 1.0})
reaction1.gene_reaction_rule = 'XylA'
model_iMM904_XAE_A1.add_reaction(reaction1)

# AdhE (r_8002) : acald_c + coa_c + nad_c ⇌ accoa_c + h_c + nadh_c
from cobra import Reaction
reaction2 = Reaction('r_8002')
reaction2.name = 'Acetaldehyde dehydrogenase'
reaction2.lower_bound = - 1000
reaction2.upper_bound = 1000
reaction2.add_metabolites({
    model_iMM904_XAE_A1.metabolites.get_by_id("acald_c"): - 1.0, 
    model_iMM904_XAE_A1.metabolites.get_by_id("coa_c"): - 1.0, 
    model_iMM904_XAE_A1.metabolites.get_by_id("nad_c"): - 1.0,
    model_iMM904_XAE_A1.metabolites.get_by_id("accoa_c"): 1.0, 
    model_iMM904_XAE_A1.metabolites.get_by_id("h_c"): 1.0, 
    model_iMM904_XAE_A1.metabolites.get_by_id("nadh_c"): 1.0})
reaction2.gene_reaction_rule = 'AdhE'
model_iMM904_XAE_A1.add_reaction(reaction2)

# ACS (r_8003) : ac_c + atp_c + coa_c → accoa_c + amp_c + ppi_c
from cobra import Reaction
reaction3 = Reaction('r_8003')
reaction3.name = 'Acetyl-CoA synthetase'
reaction3.lower_bound = 0
reaction3.upper_bound = 1000
reaction3.add_metabolites({
    model_iMM904_XAE_A1.metabolites.get_by_id("ac_c"): - 1.0, 
    model_iMM904_XAE_A1.metabolites.get_by_id("atp_c"): - 1.0, 
    model_iMM904_XAE_A1.metabolites.get_by_id("coa_c"): - 1.0,
    model_iMM904_XAE_A1.metabolites.get_by_id("accoa_c"): 1.0, 
    model_iMM904_XAE_A1.metabolites.get_by_id("amp_c"): 1.0, 
    model_iMM904_XAE_A1.metabolites.get_by_id("ppi_c"): 1.0})
reaction3.gene_reaction_rule = 'ACS'
model_iMM904_XAE_A1.add_reaction(reaction3)

# Overexpression target of TAL1 (YGR043C and YLR354C) is present in iMM904 model.  
# Overexpression target of XKS1 (YGR194C) is present in iMM904 model.  

In [73]:
# modify medium for the simulation based on Table A1. https://doi.org/10.1371/journal.pcbi.1005728
# unit (mmol/gDW/h): 1 mol of glucose every 24h to 1 gram of bacteria you might set the upper exchange flux for glucose to "1 mol / [1 gDW * 24 h]" (https://cobrapy.readthedocs.io/en/latest/media.html)

medium_iMM904_XAE_A1 = model_iMM904_XAE_A1.medium

medium_iMM904_XAE_A1["EX_ura_e"]= 1000    ### uracil[e] =>
medium_iMM904_XAE_A1["EX_ade_e"]= 0   # adenine[e] => 
medium_iMM904_XAE_A1["EX_cit_e"]= 0   # citrate[e] => 
medium_iMM904_XAE_A1["EX_gly_e"]= 0   # L-glycine[e] => 
medium_iMM904_XAE_A1["EX_ala__L_e"]= 0   # L-alanine[e] => 
medium_iMM904_XAE_A1["EX_arg__L_e"]= 0   # L-arginine[e] => 
medium_iMM904_XAE_A1["EX_asn__L_e"]= 0   # L-asparagine[e] => 
medium_iMM904_XAE_A1["EX_asp__L_e"]= 0   # L-aspartate[e] => 
medium_iMM904_XAE_A1["EX_cys__L_e"]= 0   # L-cysteine[e] => 
medium_iMM904_XAE_A1["EX_glu__L_e"]= 0   # L-glutamate[e] => 
medium_iMM904_XAE_A1["EX_gln__L_e"]= 0   # L-glutamine[e] => 
medium_iMM904_XAE_A1["EX_ile__L_e"]= 0   # L-isoleucine[e] => 
medium_iMM904_XAE_A1["EX_leu__L_e"]= 1000   ### L-leucine[e] => 
medium_iMM904_XAE_A1["EX_lys__L_e"]= 0   # L-lysine[e] => 
medium_iMM904_XAE_A1["EX_met__L_e"]= 0   # L-methionine[e] => 
medium_iMM904_XAE_A1["EX_phe__L_e"]= 0   # L-phenylalanine[e] => 
medium_iMM904_XAE_A1["EX_pro__L_e"]= 0   # L-proline[e] => 
medium_iMM904_XAE_A1["EX_ser__L_e"]= 0   # L-serine[e] => 
medium_iMM904_XAE_A1["EX_thr__L_e"]= 0   # L-threonine[e] => 
medium_iMM904_XAE_A1["EX_trp__L_e"]= 0   # L-tryptophan[e] => 
medium_iMM904_XAE_A1["EX_tyr__L_e"]= 0   # L-tyrosine[e] => 
medium_iMM904_XAE_A1["EX_val__L_e"]= 0   # L-valine[e] => 
medium_iMM904_XAE_A1["EX_ribflv_e"]= 0   # riboflavin[e] => 
medium_iMM904_XAE_A1["EX_pnto__R_e"]= 0.78   # (R)-pantothenate[e] => 
medium_iMM904_XAE_A1["EX_4abz_e"]= 0.78   # 4-aminobenzoate[e] => 
medium_iMM904_XAE_A1["EX_btn_e"]= 0.78   # biotin[e] => 
medium_iMM904_XAE_A1["EX_inost_e"]= 0.78   # myo-inositol[e] => 
medium_iMM904_XAE_A1["EX_nac_e"]= 0.78   # nicotinate[e] => 
medium_iMM904_XAE_A1["EX_thmpp_e"]= 0.78   # thiamine[e] => 
medium_iMM904_XAE_A1["EX_nh4_e"]= 1000   # ammonium[e] <=> 
medium_iMM904_XAE_A1["EX_fe2_e"]= 1000   # iron(2+)[e] <=> 
medium_iMM904_XAE_A1["EX_pi_e"]= 1000   # phosphate[e] <=> 
medium_iMM904_XAE_A1["EX_k_e"]= 1000   # potassium[e] <=> 
medium_iMM904_XAE_A1["EX_na1_e"]= 1000   # sodium[e] <=> 
medium_iMM904_XAE_A1["EX_so4_e"]= 1000   # sulphate[e] <=> 
medium_iMM904_XAE_A1["EX_h2o_e"]= 1000  # H2O
medium_iMM904_XAE_A1["EX_h_e"]= 1000  #H+
#not present: medium_iMM904_XAE_A1["r_2028"]= 0.78   # pyridoxine[e] => 
#not present: medium_iMM904_XAE_A1["r_1792"]= 0   # folate[e] => 
#not present: medium_iMM904_XAE_A1["r_4593"]= 1000   # chloride[e] <=> 
#not present: medium_iMM904_XAE_A1["r_4597"]= 1000   # Mg(2+)[e] <=> 
#not present: medium_iMM904_XAE_A1["r_4600"]= 1000   # Ca(2+)[e] <=> 
medium_iMM904_XAE_A1["EX_o2_e"]= 1000   # oxygen[e] <=>      # if anaerobic, set 0

#1) Glucose only <br>
#2) Xylose only <br>
#3) Acetate only <br>
#4) Glu + Acet <br>
#5) Xyl + Acet <br>
medium_iMM904_XAE_A1["EX_glc__D_e"]= 0   # D-glucose[e] <=> 
medium_iMM904_XAE_A1["EX_xyl__D_e"]= 20    # D-xylose[e] => 
medium_iMM904_XAE_A1["EX_ac_e"]=     2    # acetate[e] => 

model_iMM904_XAE_A1.medium = medium_iMM904_XAE_A1
model_iMM904_XAE_A1.medium

from cameo import fba
import pandas as pd

fba_result_iMM904_XAE_A1 = fba(model_iMM904_XAE_A1)
fba_result_iMM904_XAE_A1
#fba_result_iMM904_XAE_A1.data_frame
#fba_result_iMM904_XAE_A1.data_frame.to_csv('3_XAEA1_5_xyl_acet.csv')


In [72]:
# change objective

#fba_result_iMM904_XAE_A1
model_iMM904_XAE_A1.objective = "ALCD2ir" # acald_c + h_c + nadh_c --> etoh_c + nad_c
model_iMM904_XAE_A1.optimize().objective_value

999999.0

In [60]:
import ipywidgets as widgets
from IPython.display import display
import cobra
import escher

solution = model_iMM904_XAE_A1.optimize()
escher.Builder('iMM904.Central carbon metabolism', reaction_data = solution.fluxes)


Builder(reaction_data={'CITtcp': 0.0, '13BGH': 0.0, '13BGHe': 0.0, '13GS': 3.5737033277484285, '16GS': 0.0, '2…

In [173]:
import pandas as pd
from cameo import fba
from cobra.flux_analysis import flux_variability_analysis

flux_variability_analysis(model_iMM904_XAE_A1)
table_FVA_flux_XAE_A1 = flux_variability_analysis(model_iMM904_XAE_A1)
#table_FVA_flux_XAE_A1.to_csv('3_XAEA1_FVA_5_xyl_acet.csv')

# FVA (flux variability analysis) <br>
: finds the ranges of each metabolic flux at the optimum.

https://www.sciencedirect.com/science/article/pii/S0888754321000240 <br>
Flux variability analysis (FVA) determines a range of minimum and maximum fluxes of a reaction that permit optimal flux to flow through the objective function.<br>

If the difference between minimum and maximum flux values for a reaction is non-zero, it indicates that there exists a high flexibility (plasticity) for that reaction and the organism has alternate compensatory pathways for survival. <br>
Alternatively, when the difference in min and max flux values is zero, plasticity of the reaction is low and hence the reaction is crucial for organism's survival. The genes associated with these reactions are considered to be essential to the organism.<br>

In [ ]:
import pandas as pd
from cameo import fba
from cobra.flux_analysis import flux_variability_analysis

flux_variability_analysis(model_iMM904_BY4741)
table_FVA_flux_BY4741 = flux_variability_analysis(model_iMM904_BY4741)
table_FVA_flux_BY4741.data_frame
#table_FVA_flux_BY4741.data_frame.to_csv('1_FVA_4_glu_acet.csv')


flux_variability_analysis(model_iMM904_XAE)

flux_variability_analysis(model_iMM904_XAE_A1)


# Resources

In [15]:
# Absent metabolites in iMM904


4nphp_c (4-nitrophenyl phosphate) # http://bigg.ucsd.edu/universal/metabolites/4nphp

4nph_c (4-nitrophenol) # http://bigg.ucsd.edu/models/universal/metabolites/4nph

ca2_c (Calcium) # http://bigg.ucsd.edu/universal/metabolites/ca2

ca2_g (Calcium)



SyntaxError: invalid syntax (<ipython-input-15-a6620b4a757f>, line 4)

In [16]:
# PHO13 (r_8004) : h2o_c + 4nphp_c --→ h_c + pi_c + 4nph_c

from cobra import Reaction
reaction4 = Reaction('r_8004')
reaction4.name = 'Acetyl-CoA synthetase'
reaction4.lower_bound = 0
reaction4.upper_bound = 1000
reaction4.add_metabolites({
    XAE_A1.metabolites.get_by_id("s_0362[c]"): - 1.0, 
    XAE_A1.metabolites.get_by_id("s_0434[c]"): - 1.0, 
    XAE_A1.metabolites.get_by_id("s_0529[c]"): - 1.0,
    XAE_A1.metabolites.get_by_id("s_0373[c]"): 1.0, 
    XAE_A1.metabolites.get_by_id("s_0423[c]"): 1.0, 
    XAE_A1.metabolites.get_by_id("s_0633[c]"): 1.0})
reaction4.gene_reaction_rule = 'ACS'
XAE_A1.add_reaction(reaction3)

NameError: name 'XAE_A1' is not defined

In [ ]:
# PMR1 (r_8005) : h2o_g + ca2_c + atp_g --→ h_g + ca2_g + pi_g + adp_g

from cobra import Reaction
reaction4 = Reaction('r_8005')
reaction4.name = 'Acetyl-CoA synthetase'
reaction4.lower_bound = 0
reaction4.upper_bound = 1000
reaction4.add_metabolites({
    XAE_A1.metabolites.get_by_id("s_0362[c]"): - 1.0, 
    XAE_A1.metabolites.get_by_id("s_0434[c]"): - 1.0, 
    XAE_A1.metabolites.get_by_id("s_0529[c]"): - 1.0,
    XAE_A1.metabolites.get_by_id("s_0373[c]"): 1.0, 
    XAE_A1.metabolites.get_by_id("s_0423[c]"): 1.0, 
    XAE_A1.metabolites.get_by_id("s_0633[c]"): 1.0})
reaction4.gene_reaction_rule = 'ACS'
XAE_A1.add_reaction(reaction3)

In [ ]:
# ASC1 (r_8006) : 

from cobra import Reaction
reaction4 = Reaction('r_8005')
reaction4.name = 'Acetyl-CoA synthetase'
reaction4.lower_bound = 0
reaction4.upper_bound = 1000
reaction4.add_metabolites({
    XAE_A1.metabolites.get_by_id("s_0362[c]"): - 1.0, 
    XAE_A1.metabolites.get_by_id("s_0434[c]"): - 1.0, 
    XAE_A1.metabolites.get_by_id("s_0529[c]"): - 1.0,
    XAE_A1.metabolites.get_by_id("s_0373[c]"): 1.0, 
    XAE_A1.metabolites.get_by_id("s_0423[c]"): 1.0, 
    XAE_A1.metabolites.get_by_id("s_0633[c]"): 1.0})
reaction4.gene_reaction_rule = 'ACS'
XAE_A1.add_reaction(reaction3)